In [25]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
import pandas as pd
from uuid import uuid4
import os
load_dotenv()

True

In [26]:
df = pd.read_csv("dataset/wikipedia.csv")
df.head()

,id,parent_id,timestamp,hash,title,text
0,1,19472643.0,2022-04-24T00:35:51Z,oo2rvi3f9p4u9czu3opfvbsl0jp9aog,Asam deoksiribonukleat,"Asam deoksiribonukleat, lebih dikenal dengan s..."
1,3,19395623.0,2022-05-05T11:26:32Z,lq3uz0weougma29prpmkm1l5at48s4w,Anwar Sadat,Muhammad Anwar el-Sadat; ) adalah seorang poli...
2,4,14292725.0,2021-07-05T02:41:20Z,3h1z248a5ic1u3odrse27wg0o0slljl,Azhar Mansor,Datuk Azhar Mansor adalah orang Malaysia perta...
3,5,21368465.0,2022-09-26T04:25:33Z,ololfquhgvu4wzszfu1yaqo3amb8xnm,Arkeologi,Arkeologi atau ilmu kepurbakalaan adalah ilmu ...
4,6,21768805.0,2022-11-02T02:33:49Z,mkxdb4xfmgse21h6lvtoflnvwlepdb3,Antropologi,Antropologi adalah ilmu tentang manusia. Antro...


In [27]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Ukuran potongan dalam karakter
    chunk_overlap=200  # Jumlah karakter yang tumpang tindih antar potongan
)

In [65]:
documents = []
for (i, row) in df.iterrows():
    split_doc = text_splitter.split_text(row["text"])
    uuid = str(uuid4())
    for content in split_doc:
        documents.append(Document(metadata={"parent_id": uuid, "title": row["title"]}, page_content=content))

In [66]:
uuids = [str(uuid4()) for _ in documents]

In [80]:
print(f"Banyak Chunk dari {df.shape[0]} Document: {len(documents)}")
documents

Banyak Chunk dari 50 Document: 1253


[Document(metadata={'parent_id': '210e354b-8343-4a04-b21c-cdacc4cef488', 'title': 'Asam deoksiribonukleat'}, page_content='Asam deoksiribonukleat, lebih dikenal dengan singkatan DNA (bahasa Inggris: deoxyribonucleic acid), adalah salah satu jenis asam nukleat yang memiliki kemampuan pewarisan sifat. Keberadaan asam deoksiribonukleat ditemukan di dalam nukleoprotein yang membentuk inti sel. James Dewey Watson dan Francis Crick merupakan ilmuwan pertama yang mengajukan model struktur DNA pada tahun 1953 dengan bentuk pilinan ganda. Setiap DNA tersusun dari dua buah rantai polinukleotida. DNA merupakan sejenis biomolekul yang menyimpan dan menyandi instruksi-instruksi genetika setiap organisme dan banyak jenis virus. Instruksi-instruksi genetika ini berperan penting dalam pertumbuhan, perkembangan, dan fungsi organisme dan virus. DNA merupakan asam nukleat; bersamaan dengan protein dan karbohidrat, asam nukleat adalah makromolekul esensial bagi seluruh makhluk hidup yang diketahui. Kebany

In [81]:
embeddings =  HuggingFaceEmbeddings(
        model_name="LazarusNLP/all-indo-e5-small-v4",     # Provide the pre-trained model's path
        model_kwargs={'device':'cpu'}, # Pass the model configuration options
        encode_kwargs={'normalize_embeddings': False}, # Pass the encoding options
        show_progress=True
    )

In [82]:
db = FAISS.from_documents(documents=documents, embedding=embeddings, ids=uuids)

Batches: 100%|██████████| 40/40 [01:58<00:00,  2.95s/it]


In [88]:
db.docstore.search

<bound method InMemoryDocstore.search of <langchain_community.docstore.in_memory.InMemoryDocstore object at 0x7770f42cbbc0>>

In [71]:
question = "buatkan soal uraian untuk materi matematika"
resultE5Indo = db.similarity_search(question, k=10)
resultE5Indo

Batches: 100%|██████████| 1/1 [00:00<00:00, 36.42it/s]


[Document(id='7af9842b-add6-4692-b111-41cecd81d393', metadata={'parent_id': '402151c0-b1c3-434a-8de1-f1c5abfe8b84', 'title': 'Matematika'}, page_content='Matematika sangat penting di banyak bidang, termasuk ilmu alam, rekayasa, kedokteran, keuangan, ilmu komputer, dan ilmu sosial. Beberapa bidang matematika, seperti statistika dan teori permainan, dikembangkan dalam korelasi langsung dengan terapannya, dan sering dikelompokkan dengan nama matematika terapan. Bidang matematika lainnya dikembangkan secara independen dari aplikasi apapun (dan oleh karena itu disebut matematika murni), tetapi aplikasi praktis sering ditemukan kemudian. Contoh yang tepat adalah masalah faktorisasi prima, yang merujuk kepada Euklides, tetapi yang tidak memiliki aplikasi praktis sebelum digunakan dalam sistemkripto RSA (untuk keamanan jaringan komputer).'),
 Document(id='72be2105-25b6-4a17-a49b-0eb8bd6f12fb', metadata={'parent_id': '402151c0-b1c3-434a-8de1-f1c5abfe8b84', 'title': 'Matematika'}, page_content='

In [78]:
db.similarity_search(query=question, k=10, filter={"parent_id": {"$eq": "402151c0-b1c3-434a-8de1-f1c5abfe8b84"}})

Batches: 100%|██████████| 1/1 [00:00<00:00, 36.08it/s]


[Document(id='7af9842b-add6-4692-b111-41cecd81d393', metadata={'parent_id': '402151c0-b1c3-434a-8de1-f1c5abfe8b84', 'title': 'Matematika'}, page_content='Matematika sangat penting di banyak bidang, termasuk ilmu alam, rekayasa, kedokteran, keuangan, ilmu komputer, dan ilmu sosial. Beberapa bidang matematika, seperti statistika dan teori permainan, dikembangkan dalam korelasi langsung dengan terapannya, dan sering dikelompokkan dengan nama matematika terapan. Bidang matematika lainnya dikembangkan secara independen dari aplikasi apapun (dan oleh karena itu disebut matematika murni), tetapi aplikasi praktis sering ditemukan kemudian. Contoh yang tepat adalah masalah faktorisasi prima, yang merujuk kepada Euklides, tetapi yang tidak memiliki aplikasi praktis sebelum digunakan dalam sistemkripto RSA (untuk keamanan jaringan komputer).'),
 Document(id='72be2105-25b6-4a17-a49b-0eb8bd6f12fb', metadata={'parent_id': '402151c0-b1c3-434a-8de1-f1c5abfe8b84', 'title': 'Matematika'}, page_content='

In [ ]:
db.save_local("db")